In [1]:
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBClassifier
from sklearn import  metrics  
import xgboost as xgb 
from sklearn.model_selection import GridSearchCV
from os.path import exists
from sklearn.model_selection import train_test_split     
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt


%matplotlib inline

In [2]:


filepath1="Xtr.csv"
filepath2="Xts.csv"
filepath3="ytr.csv"
Xtr = np.loadtxt(filepath1, delimiter=",")
Xts = np.loadtxt(filepath2, delimiter=",")
ytr = np.loadtxt(filepath3, delimiter=",")
num_features = Xtr.shape[1]
num_samples = Xtr.shape[0]
print('unique labels: ', np.unique(ytr))
print('number of features: ', num_features)
print('number of samples: ', num_samples)

unique labels:  [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25.]
number of features:  16
number of samples:  10000


In [3]:
Xtrain,Xtest,ytrain,ytest=train_test_split(Xtr,ytr,test_size=0.05,random_state=0) 

# Then we write codes to get good parameters

The main idea is testing the first best parameter one time and then using this best parameter to get another one.

In [4]:
def modelfit(alg,X, y,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X, label=y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='mlogloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
 
    #Fit the algorithm on the data
    alg.fit(X, y,eval_metric='mlogloss')#
 
    #Predict training set:
    dtrain_predictions = alg.predict(X)  
    dtrain_predprob = alg.predict_proba(X)[:,1]
 
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4f" % np.mean(y==dtrain_predictions))
    print ('n_estimators=',cvresult.shape[0])
        

In [5]:
xgb1 = XGBClassifier(learning_rate=0.1,n_estimators=500,max_depth=5,min_child_weight=1,gamma=0,subsample=0.8,   
                   colsample_bytree=0.8,objective= 'multi:softprob',num_class=26,seed=27)  
modelfit(xgb1,Xtrain,ytrain)  

C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)



Model Report
Accuracy : 0.9997
n_estimators= 296


In [6]:
param_cv = {
  'max_depth':range(3,10,2),
  'n_estimators': range(150,350,20)
}
xgb2 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=240, max_depth=5,
min_child_weight=1, gamma=0, subsample=0.8,colsample_bytree=0.8, objective= 'multi:softprob',num_class=6,seed=27), 
 param_grid = param_cv, cv=2,verbose=10)
xgb2.fit(Xtrain,ytrain)

print("Best parameter with max_depth and _estimators: " + str(xgb2.best_params_))
print("Best score with max_depth and _estimators: " + str(xgb2.best_score_))

Fitting 2 folds for each of 40 candidates, totalling 80 fits
[CV 1/2; 1/40] START max_depth=3, n_estimators=150..............................
[17:03:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 1/2; 1/40] END max_depth=3, n_estimators=150;, score=0.810 total time=   2.8s
[CV 2/2; 1/40] START max_depth=3, n_estimators=150..............................
[17:03:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/2; 1/40] END max_depth=3, n_estimators=150;, score=0.806 total time=   2.8s
[CV 1/2; 2/40] START max_depth=3, n_estimators=170..............................
[17:03:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 1/2; 40/40] END max_depth=9, n_estimators=330;, score=0.838 total time=   9.5s
[CV 2/2; 40/40] START max_depth=9, n_estimators=330.............................
[17:12:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/2; 40/40] END max_depth=9, n_estimators=330;, score=0.831 total time=   9.4s
[17:12:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Best parameter with max_depth and _estimators: {'max_depth': 9, 'n_estimators': 170}
Best score with max_depth and _estimators: 0.8366315789473684


In [7]:
param1 = {
  'min_child_weight':range(1,4,1)}
xgb_cv2 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=210, max_depth=9,
min_child_weight=1, gamma=0, subsample=0.8,colsample_bytree=0.8, objective= 'multi:softprob',num_class=6,seed=27), 
param_grid = param1, cv=2,verbose=10)
xgb_cv2.fit(Xtrain,ytrain)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV 1/2; 1/3] START min_child_weight=1..........................................
[17:12:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 1/2; 1/3] END ...........min_child_weight=1;, score=0.840 total time=   8.7s
[CV 2/2; 1/3] START min_child_weight=1..........................................
[17:12:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/2; 1/3] END ...........min_child_weight=1;, score=0.832 total time=   6.9s
[CV 1/2; 2/3] START min_child_weight=2..........................................
[17:12:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 1/2; 3/3] END ...........min_child_weight=3;, score=0.834 total time=   6.2s
[CV 2/2; 3/3] START min_child_weight=3..........................................
[17:12:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/2; 3/3] END ...........min_child_weight=3;, score=0.828 total time=   6.2s
[17:13:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(cv=2,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     enable_categorical=False, gamma=0,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=9, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=210, n_jobs=None, num_class=6,
                                     num_parallel_tree=None,
                                     objective='multi:softprob', predictor=None,
                                     random_state=None, reg_alpha=None,
   

In [8]:
print("Best parameter with min_child_weight: " + str(xgb_cv2.best_params_))
print("Best score with min_child_weight: " + str(xgb_cv2.best_score_))

Best parameter with min_child_weight: {'min_child_weight': 1}
Best score with min_child_weight: 0.8361052631578947


In [9]:
param2 = {
 'gamma': [i / 10.0 for i in range(0, 6)] 
}
xgb_cv3 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=210, max_depth=9,
                    min_child_weight=1, gamma=0.2, subsample=0.8,colsample_bytree=0.8, objective= 'multi:softprob',num_class=26,seed=27), 
param_grid = param2, cv=5,verbose=10)
xgb_cv3.fit(Xtrain,ytrain)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5; 1/6] START gamma=0.0...................................................
[17:13:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 1/5; 1/6] END ....................gamma=0.0;, score=0.867 total time=  10.6s
[CV 2/5; 1/6] START gamma=0.0...................................................
[17:13:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/5; 1/6] END ....................gamma=0.0;, score=0.861 total time=  10.6s
[CV 3/5; 1/6] START gamma=0.0...................................................
[17:13:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 4/5; 6/6] END ....................gamma=0.5;, score=0.863 total time=  12.3s
[CV 5/5; 6/6] START gamma=0.5...................................................
[17:18:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 5/5; 6/6] END ....................gamma=0.5;, score=0.864 total time=  11.5s
[17:19:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     enable_categorical=False, gamma=0.2,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=9, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=210, n_jobs=None,
                                     num_class=26, num_parallel_tree=None,
                                     objective='multi:softprob', predictor=None,
                                     random_state=None, reg_alpha=None,


In [10]:
print("Best parameter with gamma : " + str(xgb_cv3.best_params_))
print("Best score with gamma : " + str(xgb_cv3.best_score_))

Best parameter with gamma : {'gamma': 0.4}
Best score with gamma : 0.8643157894736841


In [11]:
param3 = {  
  'subsample': [i / 10.0 for i in range(6, 10)],  
    'colsample_bytree': [i / 10.0 for i in range(6, 10)]  
}  
xgb_cv4 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=210, max_depth=9,
                    min_child_weight=1, gamma=0.1, subsample=0.8,colsample_bytree=0.8, objective= 'multi:softprob',num_class=26,seed=27), 
param_grid = param3, cv=5,verbose=10)
xgb_cv4.fit(Xtrain,ytrain)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5; 1/16] START colsample_bytree=0.6, subsample=0.6........................
[17:19:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 1/5; 1/16] END colsample_bytree=0.6, subsample=0.6;, score=0.854 total time=  10.4s
[CV 2/5; 1/16] START colsample_bytree=0.6, subsample=0.6........................
[17:19:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/5; 1/16] END colsample_bytree=0.6, subsample=0.6;, score=0.851 total time=  10.4s
[CV 3/5; 1/16] START colsample_bytree=0.6, subsample=0.6........................
[17:19:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 4/5; 16/16] END colsample_bytree=0.9, subsample=0.9;, score=0.867 total time=  12.3s
[CV 5/5; 16/16] START colsample_bytree=0.9, subsample=0.9.......................
[17:33:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 5/5; 16/16] END colsample_bytree=0.9, subsample=0.9;, score=0.872 total time=  13.4s
[17:34:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8,
                                     enable_categorical=False, gamma=0.1,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=9, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=210, n_jobs=None,
                                     num_class=26, num_parallel_tree=None,
                                     objective='multi:softprob', predictor=None,
                                     random_state=None, reg_alpha=None,


In [12]:
print("Best parameter with subsample and colsample_bytree : " + str(xgb_cv4.best_params_))
print("Best score with subsample and colsample_bytree: " + str(xgb_cv4.best_score_))

Best parameter with subsample and colsample_bytree : {'colsample_bytree': 0.8, 'subsample': 0.9}
Best score with subsample and colsample_bytree: 0.8677894736842106


In [13]:
param4 = {  
 'learning_rate':[0.03, 0.05, 0.1,0.15,0.2]  
}
xgb_cv5 = GridSearchCV(estimator = XGBClassifier(learning_rate =0.1, n_estimators=210, max_depth=9, min_child_weight=1, gamma=0.1, subsample=0.9,reg_alpha = 0.1,colsample_bytree=0.9, objective= 'multi:softprob',num_class=26,seed=27), param_grid = param4, cv=5,verbose=10)
xgb_cv5.fit(Xtrain,ytrain)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5; 1/5] START learning_rate=0.03..........................................
[17:34:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 1/5; 1/5] END ...........learning_rate=0.03;, score=0.866 total time=  13.9s
[CV 2/5; 1/5] START learning_rate=0.03..........................................
[17:34:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/5; 1/5] END ...........learning_rate=0.03;, score=0.863 total time=  13.7s
[CV 3/5; 1/5] START learning_rate=0.03..........................................
[17:34:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 4/5; 5/5] END ............learning_rate=0.2;, score=0.864 total time=  11.4s
[CV 5/5; 5/5] START learning_rate=0.2...........................................
[17:39:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 5/5; 5/5] END ............learning_rate=0.2;, score=0.864 total time=  11.4s
[17:39:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\anaconda\envs\machine_learning\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=0.9,
                                     enable_categorical=False, gamma=0.1,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=9, min_child_weight=1,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=210, n_jobs=None,
                                     num_class=26, num_parallel_tree=None,
                                     objective='multi:softprob', predictor=None,
                                     random_state=None, reg_alpha=0.1,
 

In [14]:
print("Best parameter with learning rate : " + str(xgb_cv5.best_params_))
print("Best score with learning rate: " + str(xgb_cv5.best_score_))

Best parameter with learning rate : {'learning_rate': 0.05}
Best score with learning rate: 0.8686315789473685
